#### Import files and libraries

In [1]:
! pip install sqlalchemy==1.4.46
! pip install pandasql==0.7.3
! pip install xgboost


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
sys.path.append("../")
from sales import data_holder, data_info, common
from sales.feature_engineering \
    import sku_related_features, cc_related_features,\
        date_related_features, warehouse_related_features, \
            company_related_features, stock_related_features
import pandas as pd
import pandasql as ps

pd.set_option('display.max_columns', None)

#### Dataframes generation

In [3]:
data = pd.read_csv('../data/data sales csv/sales_data.csv', delimiter=',')
data_holder.set_data(data)
data_info.preview()

Data shape:  (1162971, 29)


,FechaFactura,mes,business_unit,team,category,class,gender,tipo_producto,sku,cc,itemnumber,color_id,color_name,size,season_cl,Canal,TiendaDim,Dimensión,CostoTotal,CURRENCYCODE,AfectoFinal,CantVendida,Promedio de Mg,product_name,familia(collection),Rut_Cliente,Año,Semana,Season_fecha
0,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,NaN,NaN,NaN,NaN,NaN,Ecomm,Ecommerce,3010,$0,CLP,$2.479,1,100%,NaN,NaN,10530209-6,2023,6,SP22
1,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,NaN,NaN,NaN,NaN,NaN,Ecomm,Ecommerce,3010,$0,CLP,$0,1,NaN,NaN,NaN,13620253-7,2023,6,SP22
2,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,NaN,NaN,NaN,NaN,NaN,Ecomm,Ecommerce,3010,$0,CLP,$0,1,NaN,NaN,NaN,15170047-0,2023,6,SP22
3,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,NaN,NaN,NaN,NaN,NaN,Ecomm,Ecommerce,3010,$0,CLP,$0,1,NaN,NaN,NaN,15327358-8,2023,6,SP22
4,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,NaN,NaN,NaN,NaN,NaN,Ecomm,Ecommerce,3010,$0,CLP,$3.319,1,100%,NaN,NaN,16711666-3,2023,6,SP22


In [4]:
df_warehouses = pd.read_csv('../data/data warehouses/warehouses.csv', delimiter=';')
df_warehouses.head()

,warehouse,is_metropolitan_zone,is_inside_mall,Unnamed: 3
0,LASCONDES,1,1,NaN
1,COSTANERA,1,1,NaN
2,LADEHESA,1,1,NaN
3,MALLSPORT,1,1,NaN
4,PTOVARAS,0,0,NaN


In [5]:
df_sale_campaigns = pd.read_csv('../data/data sale campaigns/sale_campaigns.csv', delimiter=';', encoding='latin-1')
df_sale_campaigns = df_sale_campaigns.rename(columns={
    'Fecha Inicio': 'start_date',
    'Fecha Fin': 'end_date',
    'Descripción': 'campaign_name'
})
df_sale_campaigns.head()

,start_date,end_date,campaign_name
0,01-03-2019,17-03-2019,SALE
1,01-05-2019,12-05-2019,DIA DE LA MADRE
2,05-06-2019,16-06-2019,DIA DEL PADRE
3,30-08-2019,23-09-2019,SALE
4,21-02-2020,15-03-2020,SALE


#### Data preprocessing

In [6]:
data.dtypes

FechaFactura           object
mes                     int64
business_unit          object
team                   object
category               object
class                  object
gender                 object
tipo_producto          object
sku                    object
cc                     object
itemnumber             object
color_id               object
color_name             object
size                   object
season_cl              object
Canal                  object
TiendaDim              object
Dimensión               int64
CostoTotal             object
CURRENCYCODE           object
AfectoFinal            object
CantVendida             int64
Promedio de Mg         object
product_name           object
familia(collection)    object
Rut_Cliente            object
Año                     int64
Semana                  int64
Season_fecha           object
dtype: object

In [7]:
data_info.duplicates()
data_holder.delete_duplicates()
data_info.duplicates()

Duplicates in Data set: 2602,(0.22%)
Duplicates in Data set: 0,(0.0%)


We remove all the items that refer to the shipping charge

In [8]:
data_holder.remove_rows_by_values(['sku'], ['DESPACHO'])
data_holder.remove_rows_by_values(['sku'], ['ENVIO'])

data_holder.remove_rows_by_values(['Año'], [2018])
data_holder.remove_rows_by_values(['Año'], [2019])
data_info.preview()

Data shape:  (828039, 29)


,FechaFactura,mes,business_unit,team,category,class,gender,tipo_producto,sku,cc,itemnumber,color_id,color_name,size,season_cl,Canal,TiendaDim,Dimensión,CostoTotal,CURRENCYCODE,AfectoFinal,CantVendida,Promedio de Mg,product_name,familia(collection),Rut_Cliente,Año,Semana,Season_fecha
16,08-02-2023,2,LIFE OUTDOORS,M S SPORTSWEAR,TOPS,S/S TOPS,MENS,CAMISAS,42245CYPIM,42245CYPI,42245,CYPI,Chambray: Pigeon Blue,M,SP22,Ecomm,Ecommerce,3010,$10.496,CLP,$32.773,1,68%,M S DAILY SHIRT,OTROS,19061589-8,2023,6,SP22
17,08-02-2023,2,LIFE OUTDOORS,W S SPORTSWEAR,BOTTOMS,PANTS,WOMENS,PANTALONES,56566WWIBS,56566WWIB,56566,WWIB,Whole Weave: Ink Black,S,SP22,Ecomm,Ecommerce,3010,$12.740,CLP,$41.176,1,69%,W S GARDEN ISLAND PANTS,OTROS,9194419-7,2023,6,SP22
18,08-02-2023,2,MOUNTAINS,ALPINE,OUTERWEAR,OUTERWEAR JACKETS,MENS,IMPERMEABLES,85240BLKM,85240BLK,85240,BLK,Black,M,BASICO,Ecomm,Ecommerce,3010,$37.794,CLP,$104.202,1,64%,M S TORRENTSHELL 3L JKT,TORRENTSHELL,16923253-9,2023,6,SP22
19,08-02-2023,2,MOUNTAINS,ALPINE,OUTERWEAR,OUTERWEAR JACKETS,MENS,PARKAS,84701CNYL,84701CNY,84701,CNY,Classic Navy,L,SP22,Ecomm,Ecommerce,3010,$58.591,CLP,$60.252,1,3%,M S DOWN SWEATER HOODY,DOWN SWEATER,19133957-6,2023,6,SP22
20,08-02-2023,2,MOUNTAINS,ALPINE,TOPS,MIDLAYER JACKETS,MENS,CHAQUETAS,83580BLKL,83580BLK,83580,BLK,Black,L,SP22,Ecomm,Ecommerce,3010,$32.175,CLP,$91.597,1,65%,M S R1 TECHFACE JKT,TECHFACE,16711666-3,2023,6,SP22


In [9]:
df = data_holder.get_data()
print(df[df['CantVendida'] <= 0].shape)
df = df[df['CantVendida'] > 0]
data_holder.set_data(df)

(73598, 29)


In [10]:
data_holder.remove_columns_by_features([
       'FechaFactura', 'sku', 'cc', 'itemnumber', 'TiendaDim',
       'CostoTotal', 'AfectoFinal', 'CantVendida'
])
data_info.preview()

Data shape:  (754441, 8)


,FechaFactura,sku,cc,itemnumber,TiendaDim,CostoTotal,AfectoFinal,CantVendida
16,08-02-2023,42245CYPIM,42245CYPI,42245,Ecommerce,$10.496,$32.773,1
17,08-02-2023,56566WWIBS,56566WWIB,56566,Ecommerce,$12.740,$41.176,1
18,08-02-2023,85240BLKM,85240BLK,85240,Ecommerce,$37.794,$104.202,1
19,08-02-2023,84701CNYL,84701CNY,84701,Ecommerce,$58.591,$60.252,1
20,08-02-2023,83580BLKL,83580BLK,83580,Ecommerce,$32.175,$91.597,1


In [11]:
data_holder.rename_columns({
                    'FechaFactura': 'date',
                    'TiendaDim': 'warehouse',
                    'CostoTotal': 'cost',
                    'AfectoFinal': 'sale_amount', 
                    'CantVendida': 'quantity',
                    })
data_info.preview()

Data shape:  (754441, 8)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity
16,08-02-2023,42245CYPIM,42245CYPI,42245,Ecommerce,$10.496,$32.773,1
17,08-02-2023,56566WWIBS,56566WWIB,56566,Ecommerce,$12.740,$41.176,1
18,08-02-2023,85240BLKM,85240BLK,85240,Ecommerce,$37.794,$104.202,1
19,08-02-2023,84701CNYL,84701CNY,84701,Ecommerce,$58.591,$60.252,1
20,08-02-2023,83580BLKL,83580BLK,83580,Ecommerce,$32.175,$91.597,1


In [12]:
data_info.get_types()

Data types: 

date           object
sku            object
cc             object
itemnumber     object
warehouse      object
cost           object
sale_amount    object
quantity        int64
dtype: object


In [13]:
df = data_holder.get_data()   
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
df['sale_amount'] = df['sale_amount'].str.replace('[$.]', '').astype(float).astype(int)
df['sale_amount'] = df['sale_amount'].astype(int)
df['cost'] = df['cost'].str.replace('[$.]', '').astype(float).astype(int)
df['cost'] = df['cost'].astype(int)
df['quantity'] = df['quantity'].astype(int)

data_holder.set_data(df)
df.head()

C:\Users\Enrique Urrutia\AppData\Local\Temp\ipykernel_5876\1163990109.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sale_amount'] = df['sale_amount'].str.replace('[$.]', '').astype(float).astype(int)
C:\Users\Enrique Urrutia\AppData\Local\Temp\ipykernel_5876\1163990109.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cost'] = df['cost'].str.replace('[$.]', '').astype(float).astype(int)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity
16,2023-02-08,42245CYPIM,42245CYPI,42245,Ecommerce,10496,32773,1
17,2023-02-08,56566WWIBS,56566WWIB,56566,Ecommerce,12740,41176,1
18,2023-02-08,85240BLKM,85240BLK,85240,Ecommerce,37794,104202,1
19,2023-02-08,84701CNYL,84701CNY,84701,Ecommerce,58591,60252,1
20,2023-02-08,83580BLKL,83580BLK,83580,Ecommerce,32175,91597,1


Delete rows that refer to *Pop Up* warehouse 

In [14]:
df = data_holder.get_data()
print(df[df['warehouse'] == 'Pop UP'].shape)
df = df[df['warehouse'] != 'Pop UP']
data_holder.set_data(df)

(18198, 8)


Standardize *warehouses* names

In [15]:
data_holder.get_data()['warehouse'].unique()

array(['Ecommerce', 'Concepción', 'Puerto Varas', 'Temuco', 'Alerce',
       'Pucón', 'Coyhaique', 'Costanera', 'La Dehesa', 'Mall Sport',
       'Osorno', 'BuenaVentura', 'Wholesale', 'Alto Las Condes',
       'El Golf'], dtype=object)

In [16]:
warehouse_names = {
    'Alto Las Condes': 'LASCONDES', 'Mall Sport': 'MALLSPORT',
    'La Dehesa': 'LADEHESA', 'Pucón': 'PUCON', 'Temuco': 'TEMUCO',
    'Concepción': 'CONCEPCION', 'Coyhaique': 'COYHAIQUE',
    'Puerto Varas': 'PTOVARAS', 'BuenaVentura': 'BUENAVENTURA',
    'Wholesale': 'WHOLESALE', 'Ecommerce': 'ECOMMERCE',
    'Costanera': 'COSTANERA', 'El Golf': 'ELGOLF',
    'Alerce': 'ALERCE', 'Osorno': 'OSORNO'
}
df['warehouse'] = df['warehouse'].apply(lambda x: warehouse_names[x])

data_holder.set_data(df)
df['warehouse'].unique()

array(['ECOMMERCE', 'CONCEPCION', 'PTOVARAS', 'TEMUCO', 'ALERCE', 'PUCON',
       'COYHAIQUE', 'COSTANERA', 'LADEHESA', 'MALLSPORT', 'OSORNO',
       'BUENAVENTURA', 'WHOLESALE', 'LASCONDES', 'ELGOLF'], dtype=object)

#### Codificate to unit price and group data for date-sku-warehouse

In [17]:
data_holder.set_data(
    sku_related_features.add_unit_price_information(data_holder.get_data())
)
data_info.preview()

Data shape:  (736243, 9)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity,unit_price
0,2023-02-08,42245CYPIM,42245CYPI,42245,ECOMMERCE,10496,32773,1,32773
1,2023-02-08,56566WWIBS,56566WWIB,56566,ECOMMERCE,12740,41176,1,41176
2,2023-02-08,85240BLKM,85240BLK,85240,ECOMMERCE,37794,104202,1,104202
3,2023-02-08,84701CNYL,84701CNY,84701,ECOMMERCE,58591,60252,1,60252
4,2023-02-08,83580BLKL,83580BLK,83580,ECOMMERCE,32175,91597,1,91597


In [18]:
data_holder.group_data(
        ['date', 'sku', 'cc', 'itemnumber', 'warehouse'],
        {'cost' : 'mean',
         'unit_price': 'mean',
         'sale_amount' : 'max',
         'quantity' : 'sum'})
data_info.preview()

Data shape:  (676453, 9)


,date,sku,cc,itemnumber,warehouse,cost,unit_price,sale_amount,quantity
0,2020-01-01,22990NKLL,22990NKL,22990,ECOMMERCE,19211.0,41176.0,41176,1
1,2020-01-01,28756PABLALL,28756PABL,28756,ECOMMERCE,4739.0,20168.0,20168,1
2,2020-01-01,43657BLKL,43657BLK,43657,ECOMMERCE,16019.0,41176.0,41176,1
3,2020-01-01,49446BLKALL,49446BLK,49446,ECOMMERCE,4547.0,15966.0,15966,1
4,2020-01-01,52551MAMBS,52551MAMB,52551,ECOMMERCE,19730.0,53782.0,53782,1


In [19]:
data_holder.get_data().to_csv('../data/temp/base_processed_data.csv')

#### Add Stock data and missing days

In [20]:
df_sale = data_holder.get_data()

In [24]:
df_filled = pd.read_csv('../data/filled_processed_data/filled_processed_data.csv')
df_filled = df_filled[df_filled['itemnumber'].notna()]
df_filled = df_filled[df_filled['warehouse'] != 'OSORNO']

In [32]:
df_filled = df_filled[df_filled['warehouse'] != 'OSORNO']
df_filled['warehouse'].unique()

array(['ECOMMERCE', 'BUENAVENTURA', 'CONCEPCION', 'COSTANERA',
       'COYHAIQUE', 'LADEHESA', 'LASCONDES', 'MALLSPORT', 'PTOVARAS',
       'PUCON', 'TEMUCO', 'WHOLESALE', 'ELGOLF', 'ALERCE'], dtype=object)

In [35]:
data_holder.set_data(df_filled)

In [ ]:
data_holder.get_data().to_csv('../data/temp/processed_data.csv')

#### Filter CCs to use in the analysis

In [ ]:
data = pd.read_csv('../data/filled_processed_data/filled_processed_data.csv')

data['date'] = pd.to_datetime(data['date'])
data = data[data['itemnumber'].notna()]
data = data[(data['warehouse'] != 'OSORNO') & (data['warehouse'] != 'WHOLESALE')]



data_holder.set_data(data)
data_info.preview()

Get best sellers to analysis

In [ ]:
df_best_sellers_cc = pd.read_csv('../data/best_sellers_cc.csv')
df_best_sellers_cc.shape

In [ ]:
df_other = df[df['cc'].isin(df_best_sellers_cc['cc'])]
df_other.shape

Get just 10 CCs to computational limitations

In [ ]:
df_other = df[df['cc'].isin(df_best_sellers_cc[0:10]['cc'])]
df_other.shape

In [ ]:
df_other.to_csv('../data/filled_processed_data/filled_processed_best_sellers_data.csv')